In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import logging

In [ ]:
# logging level: NOTSET, DEBUG, INFO, WARNING, ERROR, CRITICAL
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# 0. Problem setting Forcasting monthly sales for tractors

* All steps together in a [Word file](https://tanthiamhuat.files.wordpress.com/2015/12/step-by-step-guide-to-forecasting-through-arima-modeling.pdf).

* Example code solved with [Jupyter Notebook in Python](http://ucanalytics.com/blogs/wp-content/uploads/2017/08/ARIMA-TimeSeries-Analysis-of-Tractor-Sales.html).

* Original code on ucanalytics.com: [part 1](http://ucanalytics.com/blogs/forecasting-time-series-analysis-manufacturing-case-study-example-part-1/), [part 2](http://ucanalytics.com/blogs/time-series-decomposition-manufacturing-case-study-example-part-2/), [part 3](http://ucanalytics.com/blogs/step-by-step-graphic-guide-to-forecasting-through-arima-modeling-in-r-manufacturing-case-study-example), [part 4](http://ucanalytics.com/blogs/step-by-step-graphic-guide-to-forecasting-through-arima-modeling-in-r-manufacturing-case-study-example/), and [part 5](http://ucanalytics.com/blogs/how-effective-is-my-marketing-budget-regression-with-arima-errors-arimax-case-study-example-part-5/). 

* Tutorials on time series: [1](https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/), [2](https://www.digitalocean.com/community/tutorials/a-guide-to-time-series-visualization-with-python-3), [3](https://www.digitalocean.com/community/tutorials/a-guide-to-time-series-forecasting-with-arima-in-python-3), [4](https://www.blackarbs.com/blog/time-series-analysis-in-python-linear-models-to-garch/11/1/2016), 
    

# 1. Load data

# 1.1. Load directly from an online .csv

In [ ]:
from io import StringIO
import requests

url = 'http://www.ucanalytics.com/blogs/wp-content/uploads/2015/06/Tractor-Sales.csv'
s = requests.get(url).text
df = pd.read_csv(StringIO(s))
df

# 1.2 Load from a local csv

In [ ]:
input_file_name = "./Tractor-Sales.csv"
df = pd.read_csv(input_file_name)
df

# 2. Data cleaning and preparation

## 2.1 Clean missing data

Check for missing data

In [ ]:
for column in df.columns:
    print(f"{column}, {sum(df[column].isnull())}")

No missing data, but that if it were needed, we would remove the nan values

In [ ]:
df = df.dropna()

Check again for missing data

In [ ]:
for column in df.columns:
    print(f"{column}, {sum(df[column].isnull())}")

## 2.2 Create a time series data frame

By creating columns related to date time

In [ ]:
df.dtypes

Type object is a string. 

2.2.1 Create the column by hand, assuming the 28th of the month.

In [ ]:
def get_date(x):
    l = x.split("-")
    return f"20{l[1]}-{l[0]}-28"

In [ ]:
df["date"] = df["Month-Year"].map(lambda x: get_date(x))
df["datetime"] = pd.to_datetime(df["date"])
df

There is an aumamatic way to do that with a Pandas method, assuming the rows are already ordered by time with one entry per month. There are two options for frequency:

* freq = "M" -> the date is set at the end of the month
* freq = "MS" -> the date is set at the start of the month

So there is an advantage over the method by hand from above, as it already knows for each month how many days there are and sets the count at the end of the month.

In [ ]:
df["datetime"] = pd.date_range(start = "2003-01-28", freq = "M", periods = len(df))
df

Check datetime has indeed the datetime64 type. 

In [ ]:
df.dtypes

Add as columns year and month as numbers (numerical variables).

In [ ]:
df["year"] = df.datetime.dt.year
df["month"] = df.datetime.dt.month
df

Add as column the month name as string (categorical variable).

In [ ]:
import calendar
df["month_name"] = df.month.map(lambda x: calendar.month_abbr[x])
df

Rename the column of the number of tractors sold to y, for faster coding in the future.

In [ ]:
df.rename(columns = {"Number of Tractor Sold": "y"}, inplace = True)
df

Keep only the subset of interesting variables and change the order as well if needed.

In [ ]:
df = df.loc[:, ["datetime", "year", "month_name", "y"]]
df

Set the datetime as index, so that the dataframe is treated like a time series.

In [ ]:
df.set_index("datetime", inplace = True)
df

# 3. Time series analysis

In [ ]:
plt.figure(figsize = (9,6))
plt.plot(df.y)
# plt.plot(df.datetime, df.y) # this would be needed if we had not set the index of the df to be datetime

The typical way to understand a time series and later predict is to decompose into three independent components that ar ethen summed, or multiplied:

* trend (across years, a linear fit)
* seasonality (across months or seasons, a sine wave fit maybe)
* irregular remainder (ideally white noise)

If the data contains many years, we can use in between trend and seasonality a fourth component, cycle, of patterns that repeat typically every 5-7 years.

* trend (across years, a linear fit)
* seasonality (across months or seasons, a sine wave fit maybe)
* cycle (trends across many years, usually 5-7 years)
* irregular remainder (ideally white noise)

In our data a cycle is non existent. 

The key idea is that it is much easier to predict each component individually.

The prediction p(t) = Trend(t) * Seasonality(t) * Remainder (t)

## 3.2 Trend

There are two steps: 

* 1. Explore visually with a rolling average what is the frequncy that gives a linear trend
* 2. Perform a statistic test to disprove the null hypothesis that there is no trend

### 3.2.1 Rolling averages

Let's study the moving averages with 4, 6, 8, 12 months. The rolling average is computed directly with a Pandas function.

In [ ]:
def plot_once(ax, df, column, nbRolling):
    ax.plot(df.index, df[column], label='Original')
    ax.plot(df.index, df[column].rolling(window=nbRolling).mean(), label = f"{str(nbRolling).zfill(2)}-Months Rolling Mean")
    ax.plot(df.index, df[column].rolling(window=nbRolling).std(), label = f"{str(nbRolling).zfill(2)}-Months Rolling Std")
    ax.set_xlabel("Years")
    ax.set_ylabel("Number of Tractors Sold")
    ax.set_title(f"{str(nbRolling).zfill(2)}-Months Moving Average")
    ax.legend(loc='best')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize = (14, 10), sharey=False, sharex=False)
column = "y"
plot_once(axes[0][0], df, column, 4)
plot_once(axes[0][1], df, column, 6)
plot_once(axes[1][0], df, column, 8)
plot_once(axes[1][1], df, column, 12)

### 3.2.2 Dickey-Fuller Test

Null Hypothesis: there is a trend, or the time series is non-stationary.
    
We calculate a test statistic and a confidence interval, we compare with a critical value. If the test statistic is less than the critical value, we can reject the null hypothesis and say the series is stationary.

In [ ]:
# Perform Dickey-Fuller test:
from statsmodels.tsa.stattools import adfuller

In [ ]:
df_test = adfuller(df.y, autolag='AIC')
df_test

In [ ]:
df_output = pd.Series(df_test[0:4], index=['Test Statistic', 'p-value', '#lags Used', 'Number of Observations Used'])
df_output

In [ ]:
for key, value in df_test[4].items():
    df_output[f"Critical Value ({key})"] = value
df_output

In [ ]:
logging.info("Results of Dickey-Fuller Test:")
logging.info(df_output)

The test statistics is not smaller than the critical values, we we can not reject the null hypothesis, so there is a trend. 

The rms of the rolling average is almost constant in time. But the mean of the rolling average is clearly increasing, so there is a trend.

## 3.3 Seasonality

First let's overlay all years to see how the number of orders varies every month. 

In [ ]:
df

### 3.3.1 Done by hand

In [ ]:
fig, ax = plt.subplots(figsize = (9,6))
for year in range(2003, 2015):
    df_current = df.loc[(df.index >= pd.to_datetime(f"{year}-Jan-01")) & (df.index <= pd.to_datetime(f"{year}-Dec-31"))] 
    ax.plot(df_current.month_name, df_current.y, label = f"{year}")
plt.legend(loc = "best")

### 3.3.2 Done by Pandas 

In [ ]:
monthly_sales_data = pd.pivot_table(df, values = "y", columns = "year", index = "month_name")
monthly_sales_data

The months used as index appear in alphabetical order, so we want to reindex with the months in the chronological order.

In [ ]:
monthly_sales_data = monthly_sales_data.reindex(index = ['Jan','Feb','Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
monthly_sales_data

In [ ]:
monthly_sales_data.plot()

In [ ]:
# https://www.geeksforgeeks.org/box-plot-in-python-using-matplotlib/
# edges are minimum (quartile of 0%) and maximum (quartile of 100%)
# blue is the first quartile (25%) and last quartile (75%)
# green is the median (50%)
monthly_sales_data.boxplot()

In [ ]:
yearly_sales_data = pd.pivot_table(df, values = "y", columns = "month_name", index = "year")
yearly_sales_data

The months are shown in alphabetical order, so let's re-order then in the chronological order. 

In [ ]:
yearly_sales_data = yearly_sales_data[['Jan','Feb','Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]
yearly_sales_data

In [ ]:
yearly_sales_data.plot()

In [ ]:
# https://www.geeksforgeeks.org/box-plot-in-python-using-matplotlib/
# edges are minimum (quartile of 0%) and maximum (quartile of 100%)
# blue is the first quartile (25%) and last quartile (75%)
# green is the median (50%)
yearly_sales_data.boxplot()

## 3.4 Remainder

The remaineder is what remains when we take off first the trend, then the seasonality. 

We can use either a multiplication, or an addition to obtain the total predictions:

* Y(t) = Trend(t) * Seasonality(t) * Remainder(t)
* Y(t) = Trend(t) + Seasonality(t) + Remainder(t)

The results are usually the same.

But be careful that this works on really clean data, like here. In real life this does not work that well, and more advanced models, like Holt-Winters seasonal method or ARIMA models are used.

## 3.5 Decomposition



In [ ]:
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf

### 3.5.1 Decomposition via multiplication

It is done for us by the statistical model library statsmodels

In [ ]:
decomposition = sm.tsa.seasonal_decompose(df.y, model='multiplicative')
fig = decomposition.plot()
fig.set_figwidth(12)
fig.set_figheight(8)
fig.suptitle('Decomposition of multiplicative time series')
plt.show()

To calculate the three components:

* 1. We fit a line (Linear Regression).

* 2. We divide y(t)/y1(t) to obtain Sesonality(t) * Residual(t). If indeed we removed the year to year trend, the values should be similar for the months across years, so we take their mean. That gives Seasonality(t), or y2(t). 

* 3. We divide (y(t)/y1(t)) / y2(t)) to obtain the Remainder(t), or y3(t).

We can predict for new values y(t) = y1(t) * y2(t) * y3(t).

### 3.5.2 Decomposition via addition

In [ ]:
decomposition = sm.tsa.seasonal_decompose(df.y, model='additive')
fig = decomposition.plot()
fig.set_figwidth(12)
fig.set_figheight(8)
fig.suptitle('Decomposition of additive time series')
plt.show()

To calculate the three components:

* 1. We fit a line (Linear Regression).

* 2. We subtract y(t)-y1(t) to obtain Sesonality(t) + Residual(t). If indeed we removed the year to year trend, the values should be similar for the months across years, so we take their mean. That gives Seasonality(t), or y2(t). 

* 3. We subract (y(t) - y1(t)) - y2(t)) to obtain the Remainder(t), or y3(t).

We can predict for new values y(t) = y1(t) + y2(t) + y3(t).

### 3.5.3 Calculate components

We choose a hybrid method where y(t) = y1(t) * y2(t) + y3(t), given that if y3 is also a multiplication factor and not well modelled as wide noise it impacts percentage-wise more, but in absolute value it affects less. 

Via a linear regression fit

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
df

In [ ]:
plt.plot(df.y)

In [ ]:
year_start = 2003
year_end = 2014 # inclusive
nb_months_per_year = 12
X = np.array([i for i in range((year_end-year_start+1)*nb_months_per_year)])
X = X.reshape(X.shape[0], 1)
X

In [ ]:
y = df.y.values
lr = LinearRegression()
regression = lr.fit(X, y)
y1 = lr.predict(X)
df["y1"] = y1
df

In [ ]:
plt.plot(y)
plt.plot(y1)

In [ ]:
df["y_div_y1"] = df.y / df.y1
df

In [ ]:
plt.plot(df.y_div_y1)

In [ ]:
monthly_sales_data = pd.pivot_table(df, values = "y_div_y1", columns = "year", index = "month_name")
monthly_sales_data

Let's create the average across years by giving up the columns variable. 

In [ ]:
monthly_sales_data = pd.pivot_table(df, values = "y_div_y1", index = "month_name", aggfunc = np.mean)
monthly_sales_data

In [ ]:
monthly_sales_data = monthly_sales_data.reindex(index = ['Jan','Feb','Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
monthly_sales_data

In [ ]:
monthly_sales_data.y_div_y1.to_dict()

In [ ]:
df["y2"] = df.month_name.map(monthly_sales_data.y_div_y1)
df

In [ ]:
df["y1_times_y2"] = df.y1 * df.y2
df

In [ ]:
plt.plot(df.y)
plt.plot(df.y1)
plt.plot(df.y1_times_y2)

In [ ]:
df["y3"] = df.y - df.y1_times_y2
df

In [ ]:
plt.plot(df.y3)

In [ ]:
plt.hist(df.y3, bins = 20)

Ideally this should be white noise, that we could model with a gaussian distribution. Let's assume it is the case, so let's get the mean and the standard deviation, and later predict by throwing a random number from a Gaussian distribution with these parameters.

In [ ]:
y3_mu = np.mean(df.y3)
y3_std = np.std(df.y3)
logging.info(f"y3_mu={y3_mu:.3f}, y3_std={y3_std:.3f}")

In [ ]:
df["y1_times_y2_plus_y3"] = df.y1 * df.y2 + df.y3
df

In [ ]:
plt.plot(df.y)
plt.plot(df.y1)
plt.plot(df.y1_times_y2)
plt.plot(df.y1_times_y2_plus_y3)

## 3.6 Predict for the next three years

Let's create a new data frame for the following three years

In [ ]:
nb_years = 3
nb_months_per_year = 12
df_predict = pd.DataFrame()
df_predict["datetime"] = pd.date_range(start = "2015-01-28", freq = "M", periods = nb_years * nb_months_per_year)
df_predict

In [ ]:
df_predict["month"] = df_predict.datetime.dt.month
df_predict["month_name"] = df_predict.month.map(lambda x: calendar.month_abbr[x])
df_predict["y"] = 0.0 # dummy value
df_predict.set_index("datetime", inplace = True) # to interpret it as a time series
df_predict

In [ ]:
X_predict = np.array([i for i in range((year_end-year_start+1)*nb_months_per_year, (year_end-year_start+1+nb_years)*nb_months_per_year, 1)])
display(X_predict.shape)
display(X_predict)
X_predict = X_predict.reshape(X_predict.shape[0], 1)
display(X_predict.shape)
display(X_predict)

In [ ]:
df_predict["y1"] = regression.predict(X_predict)
df_predict.head()

In [ ]:
df_predict["y2"] = df_predict.month_name.map(monthly_sales_data.y_div_y1)
df_predict.head()

In [ ]:
import random
df_predict["y3"] = [random.gauss(y3_mu, y3_std) for i in range(len(df_predict))]
df_predict.head()

In [ ]:
df_predict["y1_times_y2"] = df_predict.y1 * df_predict.y2
df_predict.head()

In [ ]:
df_predict["y1_times_y2_plus_y3"] = df_predict.y1 * df_predict.y2 + df_predict.y3
df_predict.head()

In [ ]:
plt.plot(df_predict.y)
plt.plot(df_predict.y1)
plt.plot(df_predict.y1_times_y2)
plt.plot(df_predict.y1_times_y2_plus_y3)

## 3.7 Overlay the original data and the predicted data

In [ ]:
df_predict.head()

In [ ]:
df_predict_2 = df_predict.loc[:, ["y", "y1", "y1_times_y2", "y1_times_y2_plus_y3"]]
df_predict_2.head()

In [ ]:
df_predict_2.shape

In [ ]:
df.head()

In [ ]:
df_initial_2 = df.loc[:, ["y", "y1", "y1_times_y2", "y1_times_y2_plus_y3"]]
df_initial_2.head()

In [ ]:
df_initial_2.shape

In [ ]:
df_all = pd.concat([df_initial_2, df_predict_2], axis = 0)
df_all

In [ ]:
df_all.shape

In [ ]:
plt.plot(df_all.y)
plt.plot(df_all.y1)
plt.plot(df_all.y1_times_y2)
plt.plot(df_all.y1_times_y2_plus_y3)

In [ ]:
plt.plot(df_all.y1)
plt.plot(df_all.y1_times_y2)
plt.plot(df_all.y1_times_y2_plus_y3)

# 4. ARIMA

In real life this simple decomposition usually does not work that well, and more advanced models are used. 

One is ARIMA. The acronym stands for:
* AR = Auto Regressive
* I = Integrated
* MA = Moving Average

An implementation for this dataset in Python can be found [here](http://ucanalytics.com/blogs/wp-content/uploads/2017/08/ARIMA-TimeSeries-Analysis-of-Tractor-Sales.html). 